In [4]:
%load_ext lab_black

import os
import time
from crewai import Agent, Task, Process, Crew
from crewai.task import TaskOutput
from dotenv import load_dotenv

from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

from pydantic import BaseModel

import warnings

load_dotenv()
warnings.filterwarnings("ignore")

The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [5]:
### ollama 
# OLLAMA_LLM_MODEL = "llama3.1:latest"
# BASE_URL = os.environ["OPENAI_API_BASE"]
# llm = ChatOpenAI(model=OLLAMA_LLM_MODEL, base_url=BASE_URL)

### Groq
GROQ_MODEL = "llama-3.1-8b-instant"
GROQ_API_KEY = os.environ["GROQ_API_KEY"]
llm = ChatGroq(model=GROQ_MODEL, api_key=GROQ_API_KEY)

In [6]:
def callback_function(output: TaskOutput):
    # Do something after the task is completed
    # Example: Send an email to the manager
    print(
        f"""
        Task completed!
        Task: {output.description}
        Output: {output.raw_output}
    """
    )


class OutputFormat(BaseModel):
    user_profile: str


#### AGENTS ####

profile_analysis_agent = Agent(
    role="Profile Analysis Agent",
    goal="""With the client profile, your task is to analyze the client's profile, including health conditions, 
    dietary preferences, lifestyle and other important features to understand their unique nutritional needs. Furthermore, provide a prediction on the metabolic rate value based on the data, 
    to understand the base calorie consumption, water consumption and the combination of carbohydrates, proteins, and fats required per day""",
    backstory="""This agent focuses on gathering and interpreting the client's profile and provide a detailed understanding of their nutritional needs. analyze your user profile, 
        taking into account your age, height, gender, and main health goals, to develop a personalized nutrition plan. 
        They will consider your current diet, frequency of eating out or processed foods, supplement use, physical activity level, and feelings after eating. 
        They will also address your water intake, sleep quality, dietary preferences or restrictions, caffeine or alcohol consumption, stress levels, and any cultural or religious dietary practices. 
        This comprehensive approach ensures that the plan is tailored to your needs, helping you achieve your health goals effectively.""",
    llm=llm,
)

diet_recommendation_agent = Agent(
    role="Diet Recommendation Agent",
    goal="""Generate a personalized and evidence-based nutrition plan tailored to the client's profile and health goals.""",
    backstory="""Use the client's profile analysis to create a balanced meal plan, considering their dietary preferences, health conditions, 
    goals and other important variable features from the profile. Focus on Brazilian foods and based on the current diet of the client.""",
    llm=llm,
)


educational_support_agent = Agent(
    role="Educational Support Agent",
    goal="Provide ongoing education and resources on healthy eating habits, nutritional guidelines, and cooking techniques.",
    backstory="""This agent focuses on educating the client about nutrition, providing tips for grocery shopping, healthy cooking, and making informed dietary choices.""",
    llm=llm,
)

nutritionist_agent = Agent(
    role="Nutritionist Expert",
    goal="Provide comprehensive and personalized nutritional support to clients by focusing on three primary areas: understanding individual needs, developing tailored nutrition plans, and offering ongoing education",
    backstory="""Synthesizes all the gathered information, creating a comprehensive nutrition and lifestyle plan in a clear and actionable format. 
        The goal is to ensure the client receives practical, easy-to-follow advice that aligns with their goals in a organized and indented Markdown format.""",
    llm=llm,
)

In [7]:
#### TASKS ####
profile_analysis_task = Task(
    description="Analyze the client's profile to understand their dietary needs, health conditions, and preferences.\nClient's profile: {user_profile}",
    expected_output="A text string with the overview, dietary needs, summary of health condition, daily calories consumption, food preferences and restrictions, and lifestyle and activity level",
    agent=profile_analysis_agent,
)

diet_recommendation_task = Task(
    description="""Create a personalized nutrition plan based on the client's profile analysis, balancing among the carbohydrates, proteins, and fats. You can use supplements or 
    any alternative foods to fit in the daily planner meal. Provide the plan using Brazilian cuisine and basic/normal foods. Use brazilian system units, like grams, liters, etc.
    Moreover, you can use the client's profile: {user_profile} and the `profile_analysis_task` to guide your answer.""",
    expected_output="""A documentation contained: Nutrition Plan, Personalized meal plan tailored to client's goals, Dietary recommendations based on health conditions, 
    - Weekly Meal Plan (3 options for each meal): Options for each day of the week and the time for each meal, containing the quantity for each portion (e.g. grams (g), kilograms (kg), milliliters (ml), etc)
        **Breakfast:** Good options for the morning\n 
        **Lunch:** Balanced meals with portion sizes\n
        **Afternoon Snacks:** Healthy snack ideas\n
        **Dinner:** Recipes and cooking tips\n 
        **Supper:** Snack ideas before sleep\n
    """,
    agent=diet_recommendation_agent,
    context=[profile_analysis_task],
)

educational_support_task = Task(
    description="Provide educational resources and guidance on healthy eating habits and nutritional guidelines. Use profile analysis to better target the response to the customer based on this.",
    expected_output="A text with Educational Resources, Nutritional guidelines, Tips for grocery shopping, Healthy cooking techniques and good sports to practice. Respond in a direct language and formal tone.",
    agent=educational_support_agent,
    context=[profile_analysis_task],
)

nutritionist_planner_task = Task(
    description="Create a daily meal plan for the user, providing educational resources and guidance on healthy eating habits and nutritional guidelines.",
    expected_output="""Synthesized the gathered information from `profile_analysis_task`, `diet_recommendation_task` and `educational_support_task` to create a 
    completed daily meal planner for the client, with all preferences, goals and specifications from the user.
    Respond in a direct language and formal tone.""",
    agent=nutritionist_agent,
    output_format="markdown",
    output_format_model=OutputFormat,
    output_format_description=(
        "The output format is a markdown file to save in a pdf file with this structure:\n"
        "1. ## Profile Analysis Summary and the enhancements to achieve the goals and healthy life.\n"
        "2. ## Daily plan, for each day and 3 options for each meal (Breakfast, Lunch, Afternoon Snacks, Dinner and Supper)\n"
        "3. ## Supplementation: List possible supplements to fit with the daily plan, with the time to consume and the days to that, like: omega 3, caffeine, magnesium, beta-alanine, etc\n"
        "3. ## Educational Resources and Guidance\n"
        "4. ## Cooking tips and how to cook some foods to this plan\n"
        "5. ## Other important information to the user based on his profile. You can adapt this header to convey important information for the user's benefit.\n"
    ),
    context=[profile_analysis_task, diet_recommendation_task, educational_support_task],
    output_file="outputs/caiobarros_planner.md",
    create_directory=True,
)

In [8]:
%%time

# Assemble the Crew
crew = Crew(
    agents=[
        profile_analysis_agent,
        diet_recommendation_agent,
        educational_support_agent,
        nutritionist_agent,
    ],
    tasks=[
        profile_analysis_task,
        diet_recommendation_task,
        educational_support_task,
        nutritionist_planner_task,
    ],
)

# Kickoff the Crew
result = crew.kickoff(
    inputs={
        "user_profile": {
            "name": "Caio",
            "age": "26",
            "height": "1.80m",
            "gender": "Male",
            "main_health_goals": "Weight loss, reduce fat percentage, and focus on reducing abdominal fat, especially on the flanks",
            "medical_conditions_or_food_allergies": "No, I don't",
            "current_diet": {
                "morning": "Wholemeal bread with two eggs and a cup of sugar-free coffee.",
                "lunch": "100g of rice, 100g of shredded chicken, and 100g of beans.",
                "afternoon_snack": "180g of low-fat yogurt, 30g of whey protein, and a fruit (grape, apple, or guava).",
                "dinner": "100g of rice, 100g of shredded chicken, and 100g of beans.",
                "supper": "Smoothie of 100g frozen strawberries with 30g powdered milk and 30g whey protein.",
            },
            "frequency_of_eating_out_or_processed_foods": "Now and then I eat sliced turkey breast.",
            "supplements_or_medications": "Whey protein, omega 3, caffeine (200mg), and creatine.",
            "physical_activity_level": "I go to the gym 5 times a week and push high lifts at a median level.",
            "feelings_after_eating": "Normal, usually not satisfied so much.",
            "water_intake": "I need to perform better to this. I would say 2 liters per day.",
            "sleep_quality_and_duration": "Among 6h and 8 hours per day.",
            "dietary_preferences_or_restrictions": "No, I don't have any preferences.",
            "alcohol_or_caffeinated_beverages_consumption": "I like to drink coffee and I also use caffeine supplement.",
            "stress_levels_and_management": "Median.",
            "cultural_or_religious_dietary_practices": "No, I don't have.",
        }
    }
)

print("############")
print(result)

############
Agent stopped due to iteration limit or time limit.
CPU times: total: 938 ms
Wall time: 8.17 s
